### 加载环境和基础设置

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import time
import logging
import glob
import subprocess
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from datetime import datetime
import shutil

# Import necessary classes from separate files
from MacGenerator import EnergySpectrum, MacFileGenerator, set_energy_decimal_places
from RootReader import RootData
from MySim import MySim # Import the new MySim class

# 切换到当前目录，并获取工作目录
try:
    os.chdir(sys.path[0])
except FileNotFoundError:
    # Handle cases where sys.path[0] might not be a valid directory (e.g., interactive environments)
    print(f"警告: 无法切换到目录 {sys.path[0]}, 使用当前工作目录。")
    pass # Continue with the current working directory

current_path = os.path.abspath(os.curdir)
# 把r'../../build' 加入到sys.path中 (确保路径正确)
build_path = os.path.abspath(os.path.join(current_path, r'../build'))
if build_path not in sys.path:
    sys.path.append(build_path)

# 定义相对路径
rel_dir_root = r'./Data/RawData/'  # ROOT files directory
rel_dir_csv  = r'./Data/CSVData/'  # CSV output directory
rel_dir_mac  = r'./Data/MacLog/'   # MAC files directory
rel_dir_log  = r'./Data/RunLog/'   # Simulation log directory
rel_dir_geant4_executable = r'../build/CompScintSim' # Geant4 executable relative path

# 获取绝对路径，末尾加上 '/'. 使用 os.path.normpath 确保路径格式一致
abs_dir_root   = os.path.normpath(os.path.join(current_path, rel_dir_root)) + os.sep
abs_dir_csv    = os.path.normpath(os.path.join(current_path, rel_dir_csv)) + os.sep
abs_dir_mac    = os.path.normpath(os.path.join(current_path, rel_dir_mac)) + os.sep
abs_dir_log    = os.path.normpath(os.path.join(current_path, rel_dir_log)) + os.sep
abs_dir_geant4_executable = os.path.normpath(os.path.join(current_path, rel_dir_geant4_executable))

print("RootDir:  ", abs_dir_root)
print("CSVDir:   ", abs_dir_csv)
print("MacDir:   ", abs_dir_mac)
print("LogDir:   ", abs_dir_log)
print("Geant4 Exe:", abs_dir_geant4_executable)

# 确保各个目录存在
for directory in [abs_dir_root, abs_dir_csv, abs_dir_mac, abs_dir_log]:
    os.makedirs(directory, exist_ok=True)

# ---- 日志配置 ----- 
# 清除已有日志处理器，避免重复添加 
log_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s') 
log_file_handler = None # Keep track of the file handler 

def setup_logging(): 
    global log_file_handler 
    # Remove existing handlers attached to the root logger 
    for handler in logging.root.handlers[:]: 
        handler.close() # Close the handler before removing 
        logging.root.removeHandler(handler) 
    log_file_handler = None # Reset tracker 

    # 配置日志记录，日志保存在 abs_dir_log 下 
    log_filename = os.path.join(abs_dir_log, "0_batch_process.log") 
    log_file_handler = logging.FileHandler(log_filename, mode='a') # Append mode 
    log_file_handler.setFormatter(log_formatter) 

    # Add the file handler 
    logging.root.addHandler(log_file_handler) 

    # Optional: Add a stream handler to also print logs to console 
    # stream_handler = logging.StreamHandler() 
    # stream_handler.setFormatter(log_formatter) 
    # logging.root.addHandler(stream_handler) 

    # Set logging level for the root logger 
    logging.root.setLevel(logging.INFO) 

    logging.info("=" * 50) 
    logging.info("初始化批处理日志 - %s", datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
    logging.info("Log file: %s", log_filename) 

setup_logging() # Initial setup 

# 全局变量，记录批处理开始前的数据目录文件列表 
global_before_files = set(os.listdir(abs_dir_root)) 

# 设置能量显示小数位数 (来自 MacGenerator) 
set_energy_decimal_places(2) 

# ---- Geant4 运行函数 ---- 

def run_geant4(dir_geant4_exe, dir_mac_file, log_file_base, retries=3): 
    """ 
    运行 Geant4 模拟，并记录详细日志。 
    - Geant4 标准输出/错误输出写入单独的模拟日志文件。 
    - 在主批处理日志中记录模拟状态、耗时、新文件等。 
    - 处理失败重试逻辑。 

    参数: 
        dir_geant4_exe (str): Geant4 可执行文件完整路径。 
        dir_mac_file (str): 要执行的 MAC 文件完整路径。 
        log_file_base (str): 模拟日志文件的基础名 (不含路径和扩展名)。 
        retries (int): 失败后的重试次数。 

    返回: 
        list: 模拟成功时生成的新 ROOT 文件列表 (相对于 abs_dir_root)。失败时返回空列表。 
    """ 
    global global_before_files 

    mac_filename = os.path.basename(dir_mac_file) 
    logging.info(f"--- 开始运行 Geant4 [MAC: {mac_filename}] ---") 
    start_time = time.time() 

    # 检查 Geant4 可执行文件是否存在 
    if not os.path.isfile(dir_geant4_exe): 
        logging.error(f"Geant4 可执行文件未找到: {dir_geant4_exe}") 
        raise FileNotFoundError(f"Geant4 可执行文件未找到: {dir_geant4_exe}") 

    # 设置模拟特定日志文件路径 
    simulation_log_file = os.path.join(abs_dir_log, f"{log_file_base}.log") 

    # 运行 Geant4 并将 stdout 和 stderr 重定向到模拟日志文件 
    process = None 
    try: 
        with open(simulation_log_file, "w") as log_f: 
            # Use Popen for better control, especially if needing timeouts later 
            cmd = [dir_geant4_exe, "-m", dir_mac_file] 
            logging.info(f"执行命令: {' '.join(cmd)}") 
            process = subprocess.Popen(cmd, stdout=log_f, stderr=subprocess.STDOUT, cwd=os.path.dirname(dir_geant4_exe)) 
            process.wait() # Wait for the process to complete 

            if process.returncode != 0: 
                logging.warning(f"Geant4 进程返回非零代码: {process.returncode}. 查看日志: {simulation_log_file}") 
            else: 
                logging.debug(f"Geant4 进程成功完成 (返回码 0)。") 

    except FileNotFoundError: 
        logging.error(f"命令执行失败: Geant4 可执行文件 '{dir_geant4_exe}' 或 MAC 文件 '{dir_mac_file}' 不存在或无权限。") 
        return [] # Indicate failure 
    except Exception as e: 
        logging.error(f"运行 Geant4 时发生未知错误: {e}", exc_info=True) 
        if process and process.poll() is None: # Check if process is still running 
            process.terminate() # Try to terminate if hung 
        return [] # Indicate failure 

    elapsed_time = time.time() - start_time 
    logging.info(f"Geant4 进程执行完毕，耗时: {elapsed_time:.2f} 秒。") 

    # 检查新生成的文件 
    after_files = set(os.listdir(abs_dir_root)) 
    new_files = list(after_files - global_before_files) 
    new_root_files = [f for f in new_files if f.endswith('.root')] 

    logging.info(f"在 {abs_dir_root} 中检测到新文件: {new_files or '无'}") 

    # 检查是否有指示失败的 '_t0' 文件 
    failed_t0_files = [f for f in new_root_files if "_t0" in f] 
    failed = bool(failed_t0_files) 

    if failed: 
        logging.error(f"模拟失败: 检测到失败标记文件 {failed_t0_files}。") 
        # 删除所有可能相关的 *_t*.root 文件 
        pattern = os.path.join(abs_dir_root, "*_t*.root") 
        deleted_count = 0 
        for file_to_delete in glob.glob(pattern): 
            if file_to_delete in after_files: # Only delete if it's potentially from this run 
                try: 
                    os.remove(file_to_delete) 
                    logging.info(f"删除失败的输出文件: {file_to_delete}") 
                    deleted_count += 1 
                except OSError as e: 
                    logging.warning(f"无法删除文件 {file_to_delete}: {e}") 

        # 更新文件列表，排除已删除的文件 
        current_files = set(os.listdir(abs_dir_root)) 
        global_before_files = current_files # Reset before retry 

        if retries > 0: 
            logging.warning(f"尝试重新运行模拟... 剩余重试次数: {retries-1}") 
            return run_geant4(dir_geant4_exe, dir_mac_file, log_file_base, retries=retries-1) 
        else: 
            logging.error("模拟失败，已达到最大重试次数。") 
            return [] # Failed after retries 
    else: 
        # 检查是否生成了预期的 ROOT 文件 (基于 MAC 文件名) 
        expected_root_basename = mac_filename.replace('.mac', '.root') 
        # Allow for potential timestamp additions if the original file existed 
        expected_root_found = any(f.startswith(mac_filename.replace('.mac', '')) and f.endswith('.root') for f in new_root_files) 

        if not new_root_files: 
            logging.warning("模拟似乎成功 (无 _t0 文件)，但未在 {abs_dir_root} 中找到新的 .root 文件。") 
        elif not expected_root_found: 
             logging.warning(f"模拟似乎成功，但找到的 ROOT 文件 {new_root_files} 与预期 ({expected_root_basename}) 不完全匹配。") 
        else: 
            logging.info(f"模拟成功！[MAC: {mac_filename}] 耗时: {elapsed_time:.2f} 秒. 新 ROOT 文件: {new_root_files}") 

        # 更新全局文件列表以便下次对比 
        global_before_files = after_files 
        return new_root_files # Return list of new root files relative to abs_dir_root 

# ---- 辅助函数：清理目录 ---- 

def clear_data_directories(): 
    """清理并重建Data目录""" 
    logging.info("开始清理数据目录...") 
    directories_to_clear = [abs_dir_root, abs_dir_csv, abs_dir_mac, abs_dir_log] 
    for directory in directories_to_clear: 
        try: 
            if os.path.exists(directory): 
                # Be cautious with rmtree! 
                if directory.rstrip(os.sep).endswith('Data'): # Basic safety check 
                    logging.warning(f"清空并重建目录: {directory}") 
                    # Close log file handler before deleting log directory 
                    if directory == abs_dir_log and log_file_handler: 
                        logging.info("关闭日志文件句柄以便删除日志目录。") 
                        log_file_handler.close() 
                        logging.root.removeHandler(log_file_handler) 

                    shutil.rmtree(directory) 
                else: 
                    logging.error(f"目录清理安全检查失败: {directory} - 跳过删除。") 
                    continue 

            os.makedirs(directory, exist_ok=True) 
            logging.info(f"已创建/确保目录存在: {directory}") 

        except Exception as e: 
            logging.error(f"清理或创建目录 {directory} 时出错: {e}", exc_info=True) 

    # Re-setup logging if the log directory was cleared 
    if not logging.root.hasHandlers(): 
         print("重新设置日志记录...") 
         setup_logging() 
    else: 
        logging.info("数据目录清理完成。") 

    # Reset global file list after clearing 
    global global_before_files 
    global_before_files = set(os.listdir(abs_dir_root)) # Update after clearing 

# ---- 批量运行函数 ---- 

def run_batch_simulations(configs, batch_size): 
    """ 
    根据提供的配置列表运行批量模拟。 

    参数: 
        configs (list): 包含多个模拟配置字典的列表。 
        batch_size (int): 每个配置要运行的模拟次数。 
    """ 
    if not isinstance(configs, list): 
        configs = [configs] # Allow single config dict 

    total_simulations = len(configs) * batch_size 
    logging.info(f"--- 开始批量模拟 ---") 
    logging.info(f"配置数量: {len(configs)}") 
    logging.info(f"每个配置的运行次数 (Batch): {batch_size}") 
    logging.info(f"总模拟运行次数: {total_simulations}") 

    global_sim_counter = 0 
    successful_runs = 0 
    failed_runs = 0 

    with tqdm(total=total_simulations, desc="运行 Geant4 模拟", unit="run") as pbar: 
        for config_index, config in enumerate(configs): 
            config_name = config.get('profile', f'config_{config_index+1}') 
            logging.info(f"==> 处理配置 {config_index + 1}/{len(configs)}: '{config_name}' ===") 

            for i in range(batch_size): 
                global_sim_counter += 1 
                run_label = f"{config_name}_batch{i+1}_run{global_sim_counter}" # Unique label for logs/files 
                pbar.set_description(f"运行: {run_label}") 
                logging.info(f"-- 运行 {global_sim_counter}/{total_simulations} ({config_name} - Batch {i + 1}/{batch_size}) --") 

                try: 
                    # 1. 创建 MySim 实例 (包含能谱生成和信息打印) 
                    # Use a dynamic profile name for each run in the batch 
                    run_profile_name = f"{config.get('profile', 'sim')}_{datetime.now().strftime('%Y%m%d%H%M%S')}_{i+1}" # Ensure uniqueness 
                    current_config = config.copy() # Avoid modifying original config 
                    current_config['profile'] = run_profile_name # Assign unique profile 

                    sim = MySim.from_config(current_config) 
                    logging.info(f"使用 MySim 实例: {sim.get_profile()}") 

                    # 2. 写入 MAC 文件 
                    mac_file_path = sim.write_mac_file(abs_dir_mac) 
                    logging.info(f"生成 MAC 文件: {os.path.basename(mac_file_path)}") 

                    # 3. 运行 Geant4 模拟 
                    # Use run_label for the simulation log file base name 
                    new_root_files = run_geant4(abs_dir_geant4_executable, mac_file_path, run_label) 

                    # 4. 处理输出的 ROOT 文件 (如果成功) 
                    if new_root_files: 
                        successful_runs += 1 
                        logging.info(f"Geant4 运行成功 ({run_label})，找到 {len(new_root_files)} 个新 ROOT 文件。") 
                        for root_file_name in new_root_files: 
                            root_file_path = os.path.join(abs_dir_root, root_file_name) 
                            try: 
                                logging.info(f"处理 ROOT 文件: {root_file_name}") 
                                root_data = RootData(root_file_path) 

                                # 生成 CSV 文件名 (基于 ROOT 文件名) 
                                csv_filename_base = os.path.splitext(root_file_name)[0] 
                                csv_path = root_data.save_simulation_data(abs_dir_csv, csv_filename_base) 
                                logging.info(f"ROOT 文件处理完成，CSV 保存至: {os.path.basename(csv_path)}") 

                            except ImportError: 
                                logging.error("无法导入 ROOT 相关模块 (uproot?)。跳过 ROOT 文件处理。请确保已安装所需库。") 
                                # Optionally break or skip further processing if ROOT is essential 
                                break # Stop processing ROOT files for this batch run if library is missing 
                            except FileNotFoundError: 
                                logging.error(f"处理失败: ROOT 文件未找到 {root_file_path} (可能已被删除或移动?)") 
                            except Exception as e: 
                                logging.error(f"处理 ROOT 文件 {root_file_name} 时出错: {e}", exc_info=True) 
                        # End loop processing root files for this sim run 
                    else: 
                        failed_runs += 1 
                        logging.error(f"Geant4 运行失败或未生成 ROOT 文件 ({run_label})。") 

                except ValueError as e: 
                    failed_runs += 1 
                    logging.error(f"配置错误 ({run_label}): {e}", exc_info=True) 
                    # Continue to next iteration if config is bad 
                except FileNotFoundError as e: 
                    failed_runs += 1 
                    logging.error(f"文件未找到错误 ({run_label}): {e}", exc_info=True) 
                    # Consider stopping if essential files (like Geant4 exe) are missing 
                    # raise # Or re-raise the exception to stop the batch 
                except Exception as e: 
                    failed_runs += 1 
                    logging.error(f"模拟运行时发生未知错误 ({run_label}): {e}", exc_info=True) 
                    # Decide whether to continue or stop based on the error 

                # 更新进度条 
                pbar.update(1) 
                # Optional short sleep to prevent overwhelming system/logs 
                # time.sleep(0.1) 
            # End loop for batch runs of a single config 
        # End loop for all configs 

    logging.info("--- 批量模拟完成 ---") 
    logging.info(f"总运行次数: {global_sim_counter}") 
    logging.info(f"成功次数: {successful_runs}") 
    logging.info(f"失败次数: {failed_runs}") 
    print(f"批量模拟完成! 成功: {successful_runs}, 失败: {failed_runs}. 详情请查看日志文件。") 


def process_all_root_files():
    """处理RawData目录下的所有.root文件并保存为CSV"""
    root_files = [f for f in os.listdir(abs_dir_root) if f.endswith('.root')]
    
    print(f"\n发现{len(root_files)}个root文件，开始处理...")
    
    with tqdm(total=len(root_files), desc="处理root文件", unit="file") as pbar:
        for root_file_name in root_files:
            try:
                root_file_path = os.path.join(abs_dir_root, root_file_name)
                root_data = RootData(root_file_path)
                
                # 生成CSV文件名（基于root文件名和时间戳）
                base_name = os.path.splitext(root_file_name)[0]
                csv_filename = f"{base_name}"
                
                # 保存CSV文件
                csv_path = root_data.save_simulation_data(abs_dir_csv, csv_filename)
                logging.info(f"处理完成: {root_file_path} -> {csv_path}")
            
            except Exception as e:
                logging.error(f"处理文件 {root_file_name} 时出错: {str(e)}")
            
            pbar.update(1)
    
    print(f"完成处理所有root文件！")

RootDir:   /home/wsl2/myGeant4/CompositeScintillatorSim/auto_python/Data/RawData/
CSVDir:    /home/wsl2/myGeant4/CompositeScintillatorSim/auto_python/Data/CSVData/
MacDir:    /home/wsl2/myGeant4/CompositeScintillatorSim/auto_python/Data/MacLog/
LogDir:    /home/wsl2/myGeant4/CompositeScintillatorSim/auto_python/Data/RunLog/
Geant4 Exe: /home/wsl2/myGeant4/CompositeScintillatorSim/build/CompScintSim
能量值将保留 2 位小数


### 示例：运行批量模拟 (新配置模式)

In [2]:
# 1. 清理旧数据 (可选) 
clear_data_directories() 

# 2. 定义模拟配置列表 

# 配置 1: Range Mode (自定义源) 
config_range = { 
    'mode': 'range',             # 模式: 能量范围 + 随机粒子数 
    'profile': 'electron_range_sim', # 文件名前缀 
    'gps_mode': 'custom',        # 使用自定义 /gps/my_source/add 
    'num_events': 20,           # Geant4 /run/beamOn 次数 

    # 电子配置 
    'E_e': [0.1, 2.0],           # 电子能量范围 [min, max] MeV 
    'delta_E_e': 0.2,            # 电子能量步长 MeV 
    'N_e_once_min': 5,           # 每个能量点单次事件最少电子数 
    'N_e_once_max': 15,          # 每个能量点单次事件最多电子数 

    # 质子配置 (可选) 
    'E_p': [5.0, 10.0], 
    'delta_E_p': 1.0, 
    'N_p_once_min': 0, 
    'N_p_once_max': 1, 
} 

# 配置 2: Weighted Mode (原生 GPS) 
config_weighted = { 
    'mode': 'weighted',          # 模式: 指定能量点 + 权重 
    'profile': 'proton_weighted_sim', # 文件名前缀 
    'gps_mode': 'native',        # 使用原生 /gps/source/add 
    'num_events': 500,           # Geant4 /run/beamOn 次数 
                                 # 注意：原生模式下，总粒子数与权重和num_events相关 

    # 电子配置
    'E_e': [0.5,1, 1.5,2], 
    'weights_e': [8,2.0, 1.0,3], 
    'N_e_once_min': 50,          # 权重最小 (1.0) 的能量点对应的基础粒子数基准 
} 

# 配置 3: Single Particle Mode (方便调试或特定测试) 
config_single = { 
    'mode': 'single', 
    'profile': 'single_gamma_1MeV', 
    'gps_mode': 'custom', # 或者 'native' 
    'num_events': 1000, 
    'particle': 'gamma', 
    'energy': 1.0,        # MeV 
    'count': 1            # 每个事件产生1个该粒子 
} 

# 配置 4: 旧版随机能谱模式 (通过嵌套配置) 
config_random_legacy = { 
    'mode': 'random', # 使用新的包装器模式 
    'profile': 'legacy_random_test', 
    'gps_mode': 'custom', # 或 'native' 
    'num_events': 50, 
    'random_config': { # 嵌套旧的随机生成器配置 
        'e_energy_min': 0.5, 
        'e_energy_max': 3.0, 
        'e_types_min': 2, 
        'e_types_max': 3, 
        'p_types_min': 1, 
        'p_types_max': 2, 
        'g_types_min': 0, # 不生成伽马 
        'g_types_max': 0, 
        'use_gamma': False, 
        'e_count_min': 10, 
        'e_count_max': 25, 
        'p_count_min': 5, 
        'p_count_max': 15, 
        # 'nums' is implicitly handled by top-level 'num_events' 
    } 
} 

# 3. 设置批量运行参数 
simulation_configs = [config_range, config_weighted, config_single, config_random_legacy] # 要运行的配置列表 
runs_per_config = 2  # 每个配置运行多少次 (Batch size) 

# 4. 运行批量模拟 
run_batch_simulations(simulation_configs, runs_per_config) 

# 5. (可选) 结束后处理所有生成的 ROOT 文件 (如果之前有失败或需要重新处理) 


process_all_root_files()

运行: electron_range_sim_batch1_run1:   0%|          | 0/8 [00:00<?, ?run/s]

警告: num_events (20) 少于推荐值100，可能影响多线程效率。
配置 'electron_range_sim_20250405170724_1':
  模式: range
  每个事件 (event) 包含的总粒子数: 98
  模拟事件数 (num_events/beamOn): 20
  总模拟粒子数 (total particles): 1960
  能谱详情:
  - 类型: e-, 能量: 0.1 MeV, 权重: 14 (对应计数: 14)
  - 类型: e-, 能量: 0.3 MeV, 权重: 14 (对应计数: 14)
  - 类型: e-, 能量: 0.5 MeV, 权重: 9 (对应计数: 9)
  - 类型: e-, 能量: 0.7 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 0.9 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 1.1 MeV, 权重: 5 (对应计数: 5)
  - 类型: e-, 能量: 1.3 MeV, 权重: 12 (对应计数: 12)
  - 类型: e-, 能量: 1.5 MeV, 权重: 9 (对应计数: 9)
  - 类型: e-, 能量: 1.7 MeV, 权重: 14 (对应计数: 14)
  - 类型: e-, 能量: 1.9 MeV, 权重: 7 (对应计数: 7)
  - 类型: proton, 能量: 6.0 MeV, 权重: 1 (对应计数: 1)
  - 类型: proton, 能量: 8.0 MeV, 权重: 1 (对应计数: 1)


运行: electron_range_sim_batch2_run2:  12%|█▎        | 1/8 [00:02<00:15,  2.22s/run]

警告: num_events (20) 少于推荐值100，可能影响多线程效率。
配置 'electron_range_sim_20250405170727_2':
  模式: range
  每个事件 (event) 包含的总粒子数: 107
  模拟事件数 (num_events/beamOn): 20
  总模拟粒子数 (total particles): 2140
  能谱详情:
  - 类型: e-, 能量: 0.1 MeV, 权重: 13 (对应计数: 13)
  - 类型: e-, 能量: 0.3 MeV, 权重: 11 (对应计数: 11)
  - 类型: e-, 能量: 0.5 MeV, 权重: 13 (对应计数: 13)
  - 类型: e-, 能量: 0.7 MeV, 权重: 11 (对应计数: 11)
  - 类型: e-, 能量: 0.9 MeV, 权重: 7 (对应计数: 7)
  - 类型: e-, 能量: 1.1 MeV, 权重: 14 (对应计数: 14)
  - 类型: e-, 能量: 1.3 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 1.5 MeV, 权重: 11 (对应计数: 11)
  - 类型: e-, 能量: 1.7 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 1.9 MeV, 权重: 13 (对应计数: 13)
  - 类型: proton, 能量: 8.0 MeV, 权重: 1 (对应计数: 1)
  - 类型: proton, 能量: 9.0 MeV, 权重: 1 (对应计数: 1)


运行: proton_weighted_sim_batch1_run3:  25%|██▌       | 2/8 [00:04<00:11,  1.99s/run]

...

警告: num_events (50) 少于推荐值100，可能影响多线程效率。
配置 'legacy_random_test_20250405170738_2':
  模式: random
  每个事件 (event) 包含的总粒子数: 82
  模拟事件数 (num_events/beamOn): 50
  总模拟粒子数 (total particles): 4100
  能谱详情:
  - 类型: e-, 能量: 0.5 MeV, 权重: 24 (对应计数: 24)
  - 类型: e-, 能量: 0.6 MeV, 权重: 25 (对应计数: 25)
  - 类型: e-, 能量: 2.0 MeV, 权重: 12 (对应计数: 12)
  - 类型: proton, 能量: 14.1 MeV, 权重: 10 (对应计数: 10)
  - 类型: proton, 能量: 18.5 MeV, 权重: 11 (对应计数: 11)


运行: legacy_random_test_batch2_run8: 100%|██████████| 8/8 [00:15<00:00,  1.91s/run]


批量模拟完成! 成功: 8, 失败: 0. 详情请查看日志文件。

发现8个root文件，开始处理...


处理root文件: 100%|██████████| 8/8 [00:00<00:00, 27.04file/s]

完成处理所有root文件！


In [3]:
clear_data_directories() 

# 配置 1: Range Mode (自定义源) 
config_range = { 
    'mode': 'range',             # 模式: 能量范围 + 随机粒子数 
    'profile': 'electron_range_sim', # 文件名前缀 
    'gps_mode': 'custom',        # 使用自定义 /gps/my_source/add 
    'num_events': 100,           # Geant4 /run/beamOn 次数 

    # 电子配置 
    'E_e': [0.1, 2.0],           # 电子能量范围 [min, max] MeV 
    'delta_E_e': 0.2,            # 电子能量步长 MeV 
    'N_e_once_min': 5,           # 每个能量点单次事件最少电子数 
    'N_e_once_max': 15,          # 每个能量点单次事件最多电子数 
} 

simulation_configs = [config_range] # 要运行的配置列表 
runs_per_config = 2  # 每个配置运行多少次 (Batch size) 

# 运行批量模拟 
run_batch_simulations(simulation_configs, runs_per_config) 

运行: electron_range_sim_batch1_run1:   0%|          | 0/8 [00:00<?, ?run/s]

配置 'electron_range_sim_20250405170950_1':
  模式: range
  每个事件 (event) 包含的总粒子数: 90
  模拟事件数 (num_events/beamOn): 100
  总模拟粒子数 (total particles): 9000
  能谱详情:
  - 类型: e-, 能量: 0.1 MeV, 权重: 5 (对应计数: 5)
  - 类型: e-, 能量: 0.3 MeV, 权重: 10 (对应计数: 10)
  - 类型: e-, 能量: 0.5 MeV, 权重: 11 (对应计数: 11)
  - 类型: e-, 能量: 0.7 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 0.9 MeV, 权重: 14 (对应计数: 14)
  - 类型: e-, 能量: 1.1 MeV, 权重: 7 (对应计数: 7)
  - 类型: e-, 能量: 1.3 MeV, 权重: 7 (对应计数: 7)
  - 类型: e-, 能量: 1.5 MeV, 权重: 10 (对应计数: 10)
  - 类型: e-, 能量: 1.7 MeV, 权重: 7 (对应计数: 7)
  - 类型: e-, 能量: 1.9 MeV, 权重: 13 (对应计数: 13)


运行: electron_range_sim_batch2_run2:  12%|█▎        | 1/8 [00:02<00:16,  2.33s/run]

配置 'electron_range_sim_20250405170952_2':
  模式: range
  每个事件 (event) 包含的总粒子数: 92
  模拟事件数 (num_events/beamOn): 100
  总模拟粒子数 (total particles): 9200
  能谱详情:
  - 类型: e-, 能量: 0.1 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 0.3 MeV, 权重: 9 (对应计数: 9)
  - 类型: e-, 能量: 0.5 MeV, 权重: 15 (对应计数: 15)
  - 类型: e-, 能量: 0.7 MeV, 权重: 11 (对应计数: 11)
  - 类型: e-, 能量: 0.9 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 1.1 MeV, 权重: 9 (对应计数: 9)
  - 类型: e-, 能量: 1.3 MeV, 权重: 11 (对应计数: 11)
  - 类型: e-, 能量: 1.5 MeV, 权重: 6 (对应计数: 6)
  - 类型: e-, 能量: 1.7 MeV, 权重: 12 (对应计数: 12)
  - 类型: e-, 能量: 1.9 MeV, 权重: 7 (对应计数: 7)


运行: proton_weighted_sim_batch1_run3:  25%|██▌       | 2/8 [00:04<00:12,  2.13s/run]

...

警告: num_events (50) 少于推荐值100，可能影响多线程效率。
配置 'legacy_random_test_20250405171002_1':
  模式: random
  每个事件 (event) 包含的总粒子数: 48
  模拟事件数 (num_events/beamOn): 50
  总模拟粒子数 (total particles): 2400
  能谱详情:
  - 类型: e-, 能量: 1.5 MeV, 权重: 16 (对应计数: 16)
  - 类型: e-, 能量: 2.5 MeV, 权重: 17 (对应计数: 17)
  - 类型: proton, 能量: 11.4 MeV, 权重: 15 (对应计数: 15)


运行: legacy_random_test_batch2_run8:  88%|████████▊ | 7/8 [00:13<00:01,  1.86s/run]

警告: num_events (50) 少于推荐值100，可能影响多线程效率。
配置 'legacy_random_test_20250405171003_2':
  模式: random
  每个事件 (event) 包含的总粒子数: 51
  模拟事件数 (num_events/beamOn): 50
  总模拟粒子数 (total particles): 2550
  能谱详情:
  - 类型: e-, 能量: 0.5 MeV, 权重: 20 (对应计数: 20)
  - 类型: e-, 能量: 2.1 MeV, 权重: 11 (对应计数: 11)
  - 类型: proton, 能量: 17.5 MeV, 权重: 11 (对应计数: 11)
  - 类型: proton, 能量: 17.8 MeV, 权重: 9 (对应计数: 9)


运行: legacy_random_test_batch2_run8: 100%|██████████| 8/8 [00:15<00:00,  1.92s/run]

批量模拟完成! 成功: 8, 失败: 0. 详情请查看日志文件。
